In [ ]:
!pip install datasets transformers py7zr sentencepiece rouge-score bert-score nltk evaluate

!pip install git+https://github.com/google-research/bleurt.git

#!wget https://storage.googleapis.com/bleurt-oss/bleurt-base-128.zip
#!unzip bleurt-base-128.zip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.1/519.1 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 113.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 27.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 120.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 85.0 MB/s eta 0:0

In [ ]:
import torch
import numpy as np
from transformers import AdamW
from datasets import load_dataset
from torch.utils.data import DataLoader
from rouge_score import rouge_scorer
from bert_score import score as bert_score
from nltk.translate.bleu_score import sentence_bleu
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from datasets import load_metric

In [ ]:
# Load the dataset
dataset = load_dataset('samsum')

Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
tokenizer = GPT2Tokenizer.from_pretrained('gpt2', padding_side='left')
model = GPT2LMHeadModel.from_pretrained('gpt2')

In [ ]:
tokenizer.add_special_tokens({'eos_token': '</s>', 'pad_token': '[PAD]'})
model.resize_token_embeddings(len(tokenizer))

In [ ]:
print(f'Size of model vocabulary: {model.config.vocab_size}')

Size of model vocabulary: 50259


In [ ]:
def process_data(dialogue, summary, tokenizer, max_length):
    input_text = dialogue + tokenizer.eos_token + summary
    tokenized = tokenizer.encode_plus(input_text, return_tensors='pt', truncation=True, max_length=max_length)
    labels = tokenized['input_ids'].clone()
    return tokenized['input_ids'][0], tokenized['attention_mask'][0], labels[0]

In [ ]:
def train(model, optimizer, train_loader, device, tokenizer_pad_token_id):
    model.train()
    total_train_loss = 0

    for batch in train_loader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]

        input_ids = input_ids[:, :-1].contiguous()
        attention_mask = attention_mask[:, :-1].contiguous()
        labels = labels[:, 1:].contiguous()
        labels[labels == tokenizer_pad_token_id] = -100

        optimizer.zero_grad()

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_train_loss += loss.item()

        loss.backward()
        optimizer.step()

    average_train_loss = total_train_loss / len(train_loader)
    return average_train_loss


In [ ]:
def validate(model, val_loader, device, tokenizer_pad_token_id):
    model.eval()
    total_val_loss = 0

    with torch.no_grad():
        for batch in val_loader:
            input_ids, attention_mask, labels = [b.to(device) for b in batch]

            input_ids = input_ids[:, :-1].contiguous()
            attention_mask = attention_mask[:, :-1].contiguous()
            labels = labels[:, 1:].contiguous()
            labels[labels == tokenizer_pad_token_id] = -100

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_val_loss += loss.item()

    average_val_loss = total_val_loss / len(val_loader)
    return average_val_loss

In [ ]:
def fine_tune(model, tokenizer, dataset, max_length=256):
    num_epochs = 5
    batch_size = 3
    learning_rate = 1e-5

    train_dataset = dataset['train']
    validation_dataset = dataset['validation']

    tokenized_inputs_train = tokenizer(train_dataset['dialogue'], truncation=True, padding='max_length', max_length=max_length, return_tensors='pt')
    tokenized_targets_train = tokenizer(train_dataset['summary'], truncation=True, padding='max_length', max_length=max_length, return_tensors='pt')

    tokenized_inputs_val = tokenizer(validation_dataset['dialogue'], truncation=True, padding='max_length', max_length=max_length, return_tensors='pt')
    tokenized_targets_val = tokenizer(validation_dataset['summary'], truncation=True, padding='max_length', max_length=max_length, return_tensors='pt')

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    train_dataset = TensorDataset(tokenized_inputs_train['input_ids'], tokenized_inputs_train['attention_mask'], tokenized_targets_train['input_ids'])
    val_dataset = TensorDataset(tokenized_inputs_val['input_ids'], tokenized_inputs_val['attention_mask'], tokenized_targets_val['input_ids'])

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    optimizer = AdamW(model.parameters(), lr=learning_rate)

    for epoch in range(num_epochs):
        average_train_loss = train(model, optimizer, train_loader, device, tokenizer.pad_token_id)
        average_val_loss = validate(model, val_loader, device, tokenizer.pad_token_id)

        print(f'Epoch {epoch + 1}/{num_epochs}')
        print(f'Average Training Loss: {average_train_loss}')
        print(f'Average Validation Loss: {average_val_loss}')

    return model


In [ ]:
# Call the function to fine-tune the model
fine_tuned_model = fine_tune(model, tokenizer, dataset, max_length=256)

Epoch 1/5
Average Training Loss: 6.480368547783294
Average Validation Loss: 5.735387786404117
Epoch 2/5
Average Training Loss: 5.637575908630595
Average Validation Loss: 5.450006142640725
Epoch 3/5
Average Training Loss: 5.3782674130245764
Average Validation Loss: 5.298047046521645
Epoch 4/5
Average Training Loss: 5.188557880518514
Average Validation Loss: 5.24362513695881
Epoch 5/5
Average Training Loss: 5.076605138456329
Average Validation Loss: 5.201459838357164


In [ ]:
fine_tuned_model.save_pretrained('fine_tuned_gpt2')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/My Drive/project_PR'

Mounted at /content/drive
/content/drive/My Drive/project_PR


In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained('gpt2', padding_side='left')
fine_tuned_model = GPT2LMHeadModel.from_pretrained('/content/drive/MyDrive/project_PR/fine_tuned_gpt2')

In [ ]:
tokenizer.add_special_tokens({'eos_token': '</s>', 'pad_token': '[PAD]'})
#fine_tuned_model.resize_token_embeddings(len(tokenizer))

0

In [ ]:
import torch
import numpy as np
from nltk.translate.bleu_score import sentence_bleu
import bert_score
from datasets import load_metric
rouge = load_metric('rouge')
bleurt_metric = load_metric('bleurt')

In [ ]:
import evaluate
bleu = evaluate.load('bleu')

In [ ]:
def generate_summaries(model, input_ids, attention_mask, target_ids, tokenizer):
    summary_ids = model.generate(input_ids,
                                 attention_mask=attention_mask,
                                 pad_token_id=tokenizer.pad_token_id,
                                 max_length=256,  # Increase this value
                                 num_beams=4,
                                 early_stopping=True)
    pred_summaries = tokenizer.batch_decode(summary_ids, skip_special_tokens=True)
    real_summaries = tokenizer.batch_decode(target_ids, skip_special_tokens=True)

    return pred_summaries, real_summaries

In [ ]:
def calculate_scores(model, dataloader, tokenizer, device):
    model.eval()

    total_scores = {
        "bert": 0,
        "bleurt": 0,
        "bleu": 0,
        "rouge1": 0,
        "rouge2": 0,
        "rougeL": 0
    }
    num_batches = 0

    with torch.no_grad():
        for batch in dataloader:
            input_ids, attention_mask, target_ids = [b.to(device) for b in batch]
            pred_summaries, real_summaries = generate_summaries(model, input_ids, attention_mask, target_ids, tokenizer)

            # Calculate scores
            total_scores = update_scores(total_scores, pred_summaries, real_summaries)
            num_batches += 1
    print(total_scores)
    total_scores['bert'] = total_scores['bert']/num_batches
    total_scores['bleurt'] = total_scores['bleurt']/num_batches
    total_scores['bleu'] = total_scores['bleu']/num_batches
    total_scores['rouge1'] = total_scores['rouge1']/num_batches
    total_scores['rouge2'] = total_scores['rouge2']/num_batches
    total_scores['rougeL'] = total_scores['rougeL']/num_batches
    # Calculate averages

    return total_scores

In [ ]:
def update_scores(total_scores, pred_summaries, real_summaries):
    # ROUGE Scores
    rouge_scores = rouge.compute(predictions=pred_summaries, references=real_summaries)
    #print(rouge_scores)
    total_scores["rouge1"] += rouge_scores['rouge1'].mid.fmeasure
    total_scores["rouge2"] += rouge_scores['rouge2'].mid.fmeasure
    total_scores["rougeL"] += rouge_scores['rougeL'].mid.fmeasure

    # BERTScore
    P, R, bert_scores = bert_score.score(pred_summaries, real_summaries, lang="en", model_type='bert-base-uncased', rescale_with_baseline=True)
    total_scores["bert"] += torch.mean(bert_scores).item()

    # BLEURT Scores
    bleurt_scores = bleurt_metric.compute(predictions=pred_summaries, references=real_summaries)['scores']
    total_scores["bleurt"] += np.mean(bleurt_scores)

    # BLEU Scores
    #bleu_scores = [sentence_bleu([ref], pred, weights=(1.0, 0, 0, 0)) for ref, pred in zip(real_summaries, pred_summaries)]
    total_scores["bleu"] += bleu.compute(references=real_summaries, predictions=pred_summaries)['bleu']

    return total_scores

In [ ]:
test_dataset = dataset['test']
tokenized_inputs_test = tokenizer(test_dataset['dialogue'], truncation=True, padding='max_length', max_length=128, return_tensors='pt')
tokenized_targets_test = tokenizer(test_dataset['summary'], truncation=True, padding='max_length', max_length=64, return_tensors='pt')
test_dataset = torch.utils.data.TensorDataset(tokenized_inputs_test['input_ids'], tokenized_inputs_test['attention_mask'], tokenized_targets_test['input_ids'])
test_loader = DataLoader(test_dataset, batch_size=3, shuffle=True)

In [ ]:
# Calculate scores on the test set
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
fine_tuned_model = fine_tuned_model.to(device)
scores = calculate_scores(fine_tuned_model, test_loader, tokenizer, device)

{'bert': 15.582605845760554, 'bleurt': -337.8904607693355, 'bleu': 2.9776354244453596, 'rouge1': 79.47175555502884, 'rouge2': 25.429657337674872, 'rougeL': 59.807525871995246}


In [ ]:
# Print out the scores
print(f'ROUGE-1 Score: {scores["rouge1"]}')
print(f'ROUGE-2 Score: {scores["rouge2"]}')
print(f'ROUGE-L Score: {scores["rougeL"]}')
print(f'BERTScore: {scores["bert"]}')
print(f'BLEURT Score: {scores["bleurt"]}')
print(f'BLEU Score: {scores["bleu"]}')


ROUGE-1 Score:  0.2911053317034023
ROUGE-2 Score:  0.09314892797683104
ROUGE-L Score:  0.21907518634430492
BERTScore:  0.0570791422921632
BLEURT Score:  -0.8437693995492072
BLEU Score:  0.010907089466832819
